In [1]:
import pybkb
from pybkb.bkb import BKB
from pybkb.fusion import fuse
from pybkb.reason.bkb import BKBReasoner
import os
import pandas as pd
import tqdm

In [2]:
df = pd.read_csv('./gimlet_midlevel_and_topics_discrete.csv')
metadata = pd.read_csv('/mnt/c/Users/c_yak/Downloads/metadata.tsv', delimiter='\t')

In [3]:
df.head()

,Unnamed: 0,episode_filename_prefix,start_time,end_time,valence,energy,tension,anger,fear,happy,sad,tender,melody,articulation,rhythm_complexity,rhythm_stability,dissonance,atonality,mode,ass
0,0,5SVUuv151DEf3K1waAxCzb,609,617,Low,High,High,Medium,Low,Low,Low,Low,High,High,High,Low,Medium,Medium,Medium,4
1,1,5SVUuv151DEf3K1waAxCzb,235,280,High,High,High,Medium,Low,Low,Low,Low,High,High,High,Medium,Medium,High,Medium,4
2,2,5SVUuv151DEf3K1waAxCzb,905,917,High,High,High,Low,Low,Medium,Low,Low,High,High,Low,High,Medium,High,Medium,4
3,3,5SVUuv151DEf3K1waAxCzb,539,546,Low,High,High,Medium,Low,Low,Medium,Low,High,High,Medium,Low,Medium,Medium,Medium,4
4,4,5SVUuv151DEf3K1waAxCzb,772,777,High,High,High,Medium,Low,Low,Medium,Low,High,High,High,Low,Medium,Medium,Medium,4


In [4]:
metadata.head()

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA
2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa
3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm
4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG


In [5]:
# View Gimlet show names
set(metadata[metadata.publisher == 'Gimlet'].show_name)

{'Chompers',
 'Conviction',
 'Crimetown',
 'Every Little Thing',
 'Gimlet Academy',
 'Heavyweight',
 'Homecoming',
 'Mogul',
 'Motherhacker',
 'Motherhood Sessions',
 'Mystery Show',
 'Reply All',
 'Sampler',
 'Sandra',
 'Science Vs',
 'Shots of Science Vs',
 'So Help Me',
 'StartUp Podcast',
 'Story Pirates',
 'Surprisingly Awesome',
 'The Cut on Tuesdays',
 'The Habitat',
 'The Horror of Dolores Roach',
 'The Nod',
 'The Pitch',
 'The Two Princes',
 'Twice Removed',
 'Uncivil',
 'Undone',
 'We Came to Win',
 'Without Fail'}

In [6]:
# List and parse bkf file names
bkf_files = os.listdir('./gimlet-bkfs')
bkf_files[0]
bkf_files_map = {}
for f in bkf_files:
    _, _, i, f1, f2 = f.split('-')
    bkf_files_map[int(i)] = f

In [7]:
# Lets look at the fused BKB of all Chompers BKFs
show_name = 'all'
bkfs = []
srcs = []
for i, row in tqdm.tqdm(metadata.iterrows(), total=len(metadata.index)):
    if row.show_name == show_name or show_name == 'all':
        try:
            f = bkf_files_map[i]
            bkfs.append(BKB.load(os.path.join('gimlet-bkfs', f)))
            _, _, i, f1, f2 = f.split('-')
            _f = f1 + '-' + f2
            srcs.append(_f)
        except:
            continue

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105360/105360 [03:40<00:00, 477.69it/s]


In [8]:
col_bkb = fuse(bkfs, [1 for _ in range(len(bkfs))], srcs, collapse=True)

In [10]:
reasoner = BKBReasoner(bkb=col_bkb)

In [ ]:
res = reasoner.update('Topic ID', evidence={'tension': 'High'})

In [9]:
col_bkb.save('gimlet_collapsed.bkb')

In [10]:
quit()